In [53]:
import json

with open("../.env/config.json", "r") as f:
    CONFIG = json.load(f)

PROJECT_ID = CONFIG["project_id"]
BUCKET_NAME = CONFIG["bucket_name"]
MODEL_NAME = CONFIG["model_name"]
LOCATION = CONFIG["location"]
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"BUCKET_NAME: {BUCKET_NAME}")
print(f"MODEL_NAME: {MODEL_NAME}")
print(f"LOCATION: {LOCATION}")

PROJECT_ID: support-ticket-442808
BUCKET_NAME: tickets-bucket-julien-us-central
MODEL_NAME: text-embedding-004
LOCATION: us-central1


In [54]:
#authentication to GCP
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../.env/service-account.json"

In [55]:
#embedding the query to compare
import google.generativeai as genai

to_compare = "Hello, I have undelivered messages"

to_compare_embedding_dict = genai.embed_content(
    model=f"models/{MODEL_NAME}", content=to_compare, task_type="clustering"
)

to_compare_embedding = to_compare_embedding_dict["embedding"]

In [56]:
import math
print(math.sqrt(sum(x**2 for x in to_compare_embedding)))

0.999999309107162


In [58]:
#finding the nearest neighbors

#preparation for vector search
from google.cloud import aiplatform_v1

API_ENDPOINT="960050135.us-central1-802291245749.vdb.vertexai.goog"
INDEX_ENDPOINT="projects/802291245749/locations/us-central1/indexEndpoints/3788631196292874240"
DEPLOYED_INDEX_ID="support_tickets_index_deployed"

#vector Search client
client_options = {
  "api_endpoint": API_ENDPOINT
}
vector_search_client = aiplatform_v1.MatchServiceClient(
  client_options=client_options,
)

#building the FindNeighborsRequest object
datapoint = aiplatform_v1.IndexDatapoint(
  feature_vector=to_compare_embedding
)

#building the query object
query = aiplatform_v1.FindNeighborsRequest.Query(
  datapoint=datapoint,
  neighbor_count=5
)

#building the request object
request = aiplatform_v1.FindNeighborsRequest(
  index_endpoint=INDEX_ENDPOINT,
  deployed_index_id=DEPLOYED_INDEX_ID,
  queries=[query]
)

#sending the request
response = vector_search_client.find_neighbors(request=request)

print(response)